In [38]:
import numpy as np
import matplotlib.pyplot as plt

In [39]:
def f(t: float, z: np.ndarray) -> np.ndarray:
    G = 6.6743e-11
    m = np.array([5.972e24, 5.972e24, 5.972e24])

    r = np.array((3,2))
    r[0,:] = z[0, :]
    r[1,:] = z[1, :]
    r[2,:] = z[2, :]

    r_dot = np.array((3,2))
    r_dot[0,:] = z[3,:]
    r_dot[1,:] = z[4,:]
    r_dot[2,:] = z[5,:]

    a = np.array((3,2))
    for i in range (0,3):
        for j in range(2,6):
            if i != j:
                a[i] += G * m[j] * (z[j] - z[i]) / np.linalg.norm(z[j] - z[i] + 1e-9) ** 3

    return np.concatenate([r_dot, a], axis=0)


z = np.array([
    [0,   0],
    [1,   2],
    [-2, -1],
    [0,   0],
    [0,   0],
    [0,   0]
])

In [40]:
# RK45-Methode
A = np.array([
    [     0,           0,          0,         0,        0,        0,   0],
    [    1/5,          0,          0,         0,        0,        0,   0],
    [    3/40,        9/40,        0,         0,        0,        0,   0],
    [   44/45,      -56/15,      32/9,        0,        0,        0,   0],
    [19372/6561, -25360/2187, 64448/6561, -212/729,     0,        0,   0],
    [ 9017/3168,   -355/33,   46732/5247,  49/176, -5103/18656,   0,   0],
    [   35/384,        0,       500/1113, 125/192, -2187/6784,  11/84, 0]
])
b_tilde = np.array([35/384,   0,  500/1113,  125/192,  -2187/6784,    11/84,    0])
b = np.array([    5179/57600, 0, 7571/16695, 393/640, -92097/339200, 187/2100, 1/40])
c = np.array([        0,     1/5,   3/10,      4/5,        8/9,         1,      1])

In [41]:
def runge_kutta(f: callable, t: float, y: np.ndarray, h: float, A: np.array, b: np.array, c: np.array) -> np.ndarray:
    s = A.shape[0]

    k = np.zeros((s,len(z)))

    for i in range (0, s):
        print(A[i, :])
        print(k[:])
        print(y)
        k[i] = f(t + c[i] * h, y + h * A[i, :] @ k[:])

    y = y + h * b[:] @ k[:]

    return y

In [42]:
def dormand_prince(f: callable, t0, T, u0, TOL):
    y = np.array([u0])
    t = np.array([t0])
    h0 = (T-t0)/10

    while t0 < T:
        y_test = runge_kutta(f, t0, u0, h0, A, b, c)
        y_tilda = runge_kutta(f, t0, u0, h0, A, b_tilde, c)
        error = np.linalg.norm(y_test - y_tilda)

        H = h0 * (TOL / error)**(1/5)

        if error > TOL:
            h0 = H
            continue

        h0 = min(H, T-t0)
        t0 += h0
        u0 = y_test
        y = np.append(y, [u0], axis=0)
        t = np.append(t, t0)

    return t, y

In [43]:
k = 9
t, u = dormand_prince(f, 0, 10, z, 10**-k)
print()
plt.figure(figsize=(10, 10))
plt.plot(t, u[:,0], marker='.', label='$r_1(t)$')
plt.plot(t, u[:,1], marker='.', label='$r_2(t)$')
plt.plot(t, u[:,2], marker='.', label='$r_2(t)$')
plt.xlabel("$t$")
plt.ylabel("$u(t)$")
plt.legend()
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.title(f"3-Körper Problem mit Toleranz: $10^{-k}$")

[0. 0. 0. 0. 0. 0. 0.]
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[ 0  0]
 [ 1  2]
 [-2 -1]
 [ 0  0]
 [ 0  0]
 [ 0  0]]


ValueError: operands could not be broadcast together with shapes (6,2) (6,) 